In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,mls,mls
band,[3],3a
commitnumber,1013f91,5014a19
conc,-,3.2e-07
dv,-,0.001
klin,-,2.22e-20
molecule,{'n2o': 3.2e-07},n2o
ng_adju,-,"[0, 0]"
ng_refs,-,"[1, 2]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.000526                    1  0.005302
0.000750        2  0.000688                    2  0.005434
0.001052        3  0.001041                    3  0.005723
0.001476        4  0.001417                    4  0.006028
0.002070        5  0.001795                    5  0.006332
0.002904        6  0.002174                    6  0.006642
0.004074        7  0.002554                    7  0.006950
0.005714        8  0.002935                    8  0.007264
0.008015        9  0.003315                    9  0.007576
0.011243       10  0.003695                   10  0.007894
0.015771       11  0.004053                   11  0.008211
0.022122       12  0.004395                   12  0.008527
0.031031       13  0.004707                   13  0.008848
0.043528       14  0.005017                   14  0.009187
0.061057       15  0.006143                   15  0.010217
0.085645       16  0.009525                   16  0.013094
0.120136       17  0.014702                   17  0.017453
0.168516       18  0.020390                   18  0.022292
0.236378       19  0.026503                   19  0.027535
0.331549       20  0.033050                   20  0.033187
0.465100       21  0.040020                   21  0.039269
0.652400       22  0.047425                   22  0.045737
0.915100       23  0.053434                   23  0.051095
1.283650       24  0.053157                   24  0.051412
1.800600       25  0.047109                   25  0.046965
2.525700       26  0.039992                   26  0.041336
3.542800       27  0.033261                   27  0.035581
4.969550       28  0.027280                   28  0.029979
6.970850       29  0.022233                   29  0.024688
9.778100       30  0.017949                   30  0.019673
13.715850      31  0.013980                   31  0.014683
19.239350      32  0.010161                   32  0.009822
26.987250      33  0.006887                   33  0.005588
37.855300      34  0.004119                   34  0.002104
53.100050      35  0.001196                   35 -0.001048
73.887500      36 -0.001897                   36 -0.003865
97.662500      37 -0.004242                   37 -0.005762
121.437500     38 -0.005814                   38 -0.006911
145.212500     39 -0.007326                   39 -0.007919
168.987500     40 -0.008400                   40 -0.008522
192.762500     41 -0.007962                   41 -0.007939
216.537500     42 -0.006223                   42 -0.006324
240.312500     43 -0.004348                   43 -0.004541
264.087500     44 -0.002584                   44 -0.002804
287.862500     45 -0.000915                   45 -0.001120
311.637500     46  0.000680                   46  0.000514
335.412500     47  0.002213                   47  0.002105
359.187500     48  0.003692                   48  0.003651
382.962500     49  0.005123                   49  0.005155
406.737500     50  0.006502                   50  0.006613
430.512500     51  0.007826                   51  0.008016
454.287500     52  0.009089                   52  0.009364
478.062500     53  0.010293                   53  0.010658
501.837500     54  0.011443                   54  0.011901
525.612500     55  0.012554                   55  0.013094
549.387500     56  0.013636                   56  0.014256
573.162500     57  0.014695                   57  0.015393
596.937500     58  0.015737                   58  0.016508
620.712500     59  0.016763                   59  0.017605
644.487500     60  0.017778                   60  0.018680
668.262500     61  0.018779                   61  0.019736
692.037500     62  0.019770                   62  0.020776
715.812500     63  0.020753                   63  0.021804
739.587500     64  0.021730                   64  0.022818
763.362500     65  0.022697                   65  0.023813
78

# Fluxes by Layer

CRD                          CLIRAD                \
                        flug      fldg      fnetg       flug          fldg   
pressure    level                                                            
0.000000    1     -35.017091  0.000000 -35.017091 -35.064266  4.359594e-08   
0.000624    2     -35.017093  0.000001 -35.017091 -35.064267  1.143221e-06   
0.000876    3     -35.017093  0.000002 -35.017091 -35.064267  1.588425e-06   
0.001229    4     -35.017094  0.000003 -35.017091 -35.064267  2.219006e-06   
0.001723    5     -35.017095  0.000004 -35.017091 -35.064268  3.112699e-06   
0.002417    6     -35.017097  0.000005 -35.017091 -35.064269  4.378875e-06   
0.003391    7     -35.017099  0.000007 -35.017090 -35.064270  6.173187e-06   
0.004757    8     -35.017101  0.000010 -35.017091 -35.064271  8.715684e-06   
0.006672    9     -35.017104  0.000015 -35.017090 -35.064273  1.231848e-05   
0.009359    10    -35.017110  0.000021 -35.017089 -35.064276  1.742312e-05   
0.013128    11    -35.017117  0.000030 -35.017087 -35.064280  2.465648e-05   
0.018415    12    -35.017127  0.000042 -35.017085 -35.064285  3.490462e-05   
0.025830    13    -35.017140  0.000060 -35.017081 -35.064292  4.942207e-05   
0.036232    14    -35.017159  0.000084 -35.017075 -35.064301  6.998884e-05   
0.050823    15    -35.017186  0.000119 -35.017066 -35.064315  9.914253e-05   
0.071291    16    -35.017221  0.000170 -35.017051 -35.064332  1.413277e-04   
0.100000    17    -35.017266  0.000246 -35.017019 -35.064352  2.055427e-04   
0.140271    18    -35.017315  0.000366 -35.016949 -35.064369  3.063155e-04   
0.196760    19    -35.017365  0.000553 -35.016813 -35.064378  4.643157e-04   
0.275997    20    -35.017405  0.000842 -35.016564 -35.064366  7.111936e-04   
0.387100    21    -35.017417  0.001289 -35.016128 -35.064314  1.095379e-03   
0.543100    22    -35.017363  0.001974 -35.015389 -35.064185  1.692138e-03   
0.761700    23    -35.017181  0.003019 -35.014161 -35.063922  2.613529e-03   
1.068500    24    -35.016790  0.004570 -35.012219 -35.063456  4.004865e-03   
1.498800    25    -35.016201  0.006691 -35.009510 -35.062792  5.961416e-03   
2.102400    26    -35.015510  0.009369 -35.006141 -35.062009  8.537499e-03   
2.949000    27    -35.014791  0.012660 -35.002131 -35.061176  1.185046e-02   
4.136600    28    -35.014125  0.016674 -34.997452 -35.060383  1.606314e-02   
5.802500    29    -35.013647  0.021578 -34.992068 -35.059776  2.137230e-02   
8.139200    30    -35.013558  0.027644 -34.985914 -35.059586  2.801676e-02   
11.417000   31    -35.014209  0.035263 -34.978945 -35.060196  3.626639e-02   
16.014700   32    -35.016239  0.044908 -34.971331 -35.062293  4.636186e-02   
22.464000   33    -35.020735  0.057166 -34.963568 -35.066992  5.856456e-02   
31.510500   34    -35.029189  0.073002 -34.956187 -35.075868  7.342873e-02   
44.200100   35    -35.043769  0.093773 -34.949996 -35.091221  9.194553e-02   
62.000000   36    -35.068290  0.120815 -34.947474 -35.116872  1.153866e-01   
85.775000   37    -35.106918  0.154101 -34.952817 -35.156645  1.442737e-01   
109.550000  38    -35.150422  0.185658 -34.964765 -35.200783  1.721828e-01   
133.325000  39    -35.197626  0.216489 -34.981138 -35.248122  2.000554e-01   
157.100000  40    -35.248429  0.246660 -35.001769 -35.298360  2.279883e-01   
180.875000  41    -35.302298  0.276873 -35.025426 -35.350839  2.564647e-01   
204.650000  42    -35.357121  0.309272 -35.047850 -35.403803  2.870695e-01   
228.425000  43    -35.410858  0.345482 -35.065375 -35.455660  3.211154e-01   
252.200000  44    -35.462979  0.385357 -35.077621 -35.506004  3.586692e-01   
275.975000  45    -35.513391  0.428493 -35.084898 -35.554735  3.995011e-01   
299.750000  46    -35.562055  0.474581 -35.087474 -35.601793  4.434032e-01   
323.525000  47    -35.608943  0.523382 -35.085560 -35.647139  4.901986e-01   
347.300000  48    -35.654033  0.574705 -35.079328 -35.690723  5.397123e-01   
371.075000  49    -35.697312  0.628383 -35.0689

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')